## API활용한대기질preprocessing

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
from urllib import request
from urllib.request import urlopen
from urllib.parse import urlencode, quote_plus
from urllib.parse import unquote

In [ ]:
from datetime import datetime
from datetime import timedelta

In [ ]:
date=[]
start = datetime(2016, 1, 1)
for _ in range(1461):
    date.append(start.strftime("%Y%m%d"))
    start=start+timedelta(days=1)

In [ ]:
data=[]
for j in date:
    # 인증키 발급 필요
    url='http://openAPI.seoul.go.kr:8088/(인증키)/xml/DailyAverageCityAir/1/100/'+j
    req=requests.get(url)
    html=req.text
    soup=BeautifulSoup(html, 'html.parser')
    data.append(soup.find_all('row'))

In [ ]:
data=sum(data,[])

In [ ]:
msrdt_del=[]
msrrgn_nml=[]
msrste_nml=[]
no2l=[]
o3l=[]
col=[]
so2l=[]
pm10l=[]
pm25l=[]

In [ ]:
len(data)

34794

In [ ]:
data[0:2]

[<row>
 <msrdt_de>20160101</msrdt_de>
 <msrrgn_nm>도심권</msrrgn_nm>
 <msrste_nm>중구</msrste_nm>
 <pm10>61</pm10>
 <pm25>35</pm25>
 <o3>0.005</o3>
 <no2>0.065</no2>
 <co>0.8</co>
 <so2>0.007</so2>
 </row>,
 <row>
 <msrdt_de>20160101</msrdt_de>
 <msrrgn_nm>도심권</msrrgn_nm>
 <msrste_nm>종로구</msrste_nm>
 <pm10>68</pm10>
 <pm25>51</pm25>
 <o3>0.005</o3>
 <no2>0.047</no2>
 <co>0.7</co>
 <so2>0.006</so2>
 </row>]

In [ ]:
for i in range(len(data)):
    msrdt_del.append(data[i].find('msrdt_de').text)
    msrrgn_nml.append(data[i].find('msrrgn_nm').text)
    msrste_nml.append(data[i].find('msrste_nm').text)
    no2l.append(data[i].find('no2').text)
    o3l.append(data[i].find('o3').text)
    col.append(data[i].find('co').text)
    so2l.append(data[i].find('so2').text)
    pm10l.append(data[i].find('pm10').text)
    pm25l.append(data[i].find('pm25').text)

In [ ]:
air=pd.DataFrame()
air['msrdt']=msrdt_del
air['msrrgn_nm']=msrrgn_nml
air['msrste_nm']=msrste_nml
air['no2']=no2l
air['o3']=o3l
air['co']=col
air['so2']=so2l
air['pm10']=pm10l
air['pm25']=pm25l

In [ ]:
# 누락된 날짜
null_date=set(date)-set(air.msrdt.unique())

In [ ]:
len(null_date)

69

In [ ]:
# 누락된 날짜는 시간대별 평균을 사용해 일 평균을 구함
data2=[]
for i in null_date:
    # 인증키 발급 필요
    url='http://openAPI.seoul.go.kr:8088/(인증키)/xml/TimeAverageAirQuality/1/1000/'+ i
    req=requests.get(url)
    html=req.text
    soup=BeautifulSoup(html, 'html.parser')
    data2.append(soup.find_all('row'))

In [ ]:
len(data2)

69

In [ ]:
data2=sum(data2,[])

In [ ]:
msrdt_del=[]
msrrgn_nml=[]
msrste_nml=[]
no2l=[]
o3l=[]
col=[]
so2l=[]
pm10l=[]
pm25l=[]

In [ ]:
len(data2)

41025

In [ ]:
for i in range(len(data2)):
    msrdt_del.append(data2[i].find('msrdt').text)
    msrste_nml.append(data2[i].find('msrste_nm').text)
    no2l.append(data2[i].find('no2').text)
    o3l.append(data2[i].find('o3').text)
    col.append(data2[i].find('co').text)
    so2l.append(data2[i].find('so2').text)
    pm10l.append(data2[i].find('pm10').text)
    pm25l.append(data2[i].find('pm25').text)

In [ ]:
air_miss=pd.DataFrame()
air_miss['msrdt']=msrdt_del
air_miss['msrste_nm']=msrste_nml
air_miss['no2']=no2l
air_miss['o3']=o3l
air_miss['co']=col
air_miss['so2']=so2l
air_miss['pm10']=pm10l
air_miss['pm25']=pm25l

In [ ]:
air

,msrdt,msrrgn_nm,msrste_nm,no2,o3,co,so2,pm10,pm25
0,20160101,도심권,중구,0.065,0.005,0.8,0.007,61,35
1,20160101,도심권,종로구,0.047,0.005,0.7,0.006,68,51
2,20160101,도심권,용산구,0.046,0.005,0.7,0.005,59,46
3,20160101,서북권,은평구,0.039,0.01,0.7,0.007,58,35
4,20160101,서북권,서대문구,0.06,0.007,0.9,0.005,69,39
...,...,...,...,...,...,...,...,...,...
34789,20191231,서남권,양천구,0.017,0.019,0.4,0.004,29,17
34790,20191231,동남권,강남구,0.018,0.021,0.4,0.003,22,15
34791,20191231,동남권,서초구,0.019,0.028,0.2,0.003,28,13
34792,20191231,동남권,송파구,0.022,0.016,0.4,0.003,25,17


In [ ]:
air_miss

,msrdt,msrste_nm,no2,o3,co,so2,pm10,pm25
0,201806080700,강남구,0.013,0.021,0.3,0.004,,21
1,201806082100,강남구,0.045,0.015,0.5,0.005,43,32
2,201806080300,강남구,0.012,0.022,0.3,0.005,19,14
3,201806082200,강남구,0.04,0.013,0.5,0.005,44,34
4,201806082300,강남구,0.036,0.014,0.5,0.005,49,36
...,...,...,...,...,...,...,...,...
41020,201803310400,중랑구,0.028,0.013,0.7,0.005,55,37
41021,201803310300,중랑구,0.029,0.013,0.7,0.006,58,40
41022,201803310200,중랑구,0.036,0.007,0.8,0.006,58,42
41023,201803310100,중랑구,0.039,0.005,0.8,0.006,60,42


In [ ]:
# 일평균으로 구하기 위해 시간 부분 자르기
air_miss.msrdt=air_miss.msrdt.apply(lambda a: a[:-4])

In [ ]:
air_miss

,msrdt,msrste_nm,no2,o3,co,so2,pm10,pm25
0,20180608,강남구,0.013,0.021,0.3,0.004,,21
1,20180608,강남구,0.045,0.015,0.5,0.005,43,32
2,20180608,강남구,0.012,0.022,0.3,0.005,19,14
3,20180608,강남구,0.04,0.013,0.5,0.005,44,34
4,20180608,강남구,0.036,0.014,0.5,0.005,49,36
...,...,...,...,...,...,...,...,...
41020,20180331,중랑구,0.028,0.013,0.7,0.005,55,37
41021,20180331,중랑구,0.029,0.013,0.7,0.006,58,40
41022,20180331,중랑구,0.036,0.007,0.8,0.006,58,42
41023,20180331,중랑구,0.039,0.005,0.8,0.006,60,42


In [ ]:
air_miss[['no2','o3','co','so2','pm10','pm25']]=air_miss.iloc[:,2:].apply(lambda a: pd.to_numeric(a))

In [ ]:
del air['msrrgn_nm']

In [ ]:
air_day=air_miss.groupby(['msrdt','msrste_nm']).mean().reset_index()

In [ ]:
# air_day = 시간별 평균으로 구한 일 평균 대기질
air_day.isnull().sum()

msrdt         0
msrste_nm     0
no2          22
o3           24
co           27
so2          24
pm10         11
pm25          9
dtype: int64

In [ ]:
air_plus_miss=pd.concat([air,air_day])

In [ ]:
# 측정소별 권역코드
code=pd.read_csv('code.csv',encoding='utf-8')

In [ ]:
code.tail(3)

,권역코드,측정소명
22,104,강동구
23,104,서초구
24,104,송파구


In [ ]:
air_day=pd.merge(air_plus_miss,code,how='left',left_on='msrste_nm',right_on='측정소명').iloc[:,:-1]

In [ ]:
air_day[['no2','o3','co','so2','pm10','pm25']]=air_day.iloc[:,2:-1].apply(lambda a: round(pd.to_numeric(a),3))

In [ ]:
air_day

,msrdt,msrste_nm,no2,o3,co,so2,pm10,pm25,권역코드
0,20160101,중구,0.065,0.005,0.800,0.007,61.000,35.000,100
1,20160101,종로구,0.047,0.005,0.700,0.006,68.000,51.000,100
2,20160101,용산구,0.046,0.005,0.700,0.005,59.000,46.000,100
3,20160101,은평구,0.039,0.010,0.700,0.007,58.000,35.000,101
4,20160101,서대문구,0.060,0.007,0.900,0.005,69.000,39.000,101
...,...,...,...,...,...,...,...,...,...
36514,20190908,용산구,0.020,0.031,0.446,0.004,29.833,22.667,100
36515,20190908,은평구,0.014,0.039,0.604,0.004,39.292,25.458,101
36516,20190908,종로구,0.015,0.030,0.454,0.003,32.542,20.750,100
36517,20190908,중구,0.016,0.035,NaN,0.003,33.208,19.750,100


In [ ]:
# air_code=일평균 대기질을 구하지 못한 구에 대한 대체값으로 사용할 권역별 평균 대기질
air_code=air_day.groupby(['msrdt','권역코드']).mean().reset_index()

In [ ]:
air_code

,msrdt,권역코드,no2,o3,co,so2,pm10,pm25
0,20160101,100,0.052667,0.005000,0.733333,0.006000,62.666667,44.000000
1,20160101,101,0.047000,0.008333,0.866667,0.006000,62.000000,41.000000
2,20160101,102,0.049625,0.004250,1.062500,0.005875,72.125000,46.875000
3,20160101,103,0.044857,0.007714,1.014286,0.005571,66.857143,45.142857
4,20160101,104,0.043500,0.004500,0.900000,0.005500,72.500000,42.500000
...,...,...,...,...,...,...,...,...
7300,20191231,100,0.014000,0.021333,0.400000,0.002667,22.333333,17.333333
7301,20191231,101,0.013333,0.022000,0.433333,0.002333,23.000000,11.333333
7302,20191231,102,0.015750,0.023000,0.387500,0.002750,24.375000,13.875000
7303,20191231,103,0.016429,0.021143,0.357143,0.003143,25.285714,16.000000


In [ ]:
air_day.isnull().sum()

msrdt         0
msrste_nm     0
no2          22
o3           24
co           27
so2          24
pm10         11
pm25          9
권역코드          0
dtype: int64

In [ ]:
for j in air_day.columns:
    index=np.where(air_day[j].isnull())[0]
    for i in index:
        air_day[j][i]=air_code[(air_code.msrdt==air_day.msrdt[i]) & (air_code.권역코드==air_day.권역코드[i])][j]

C:\Users\shs40\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
air_day.isnull().sum()

msrdt        0
msrste_nm    0
no2          0
o3           0
co           0
so2          0
pm10         0
pm25         0
권역코드         0
dtype: int64

In [ ]:
air_day

,msrdt,msrste_nm,no2,o3,co,so2,pm10,pm25,권역코드
0,20160101,중구,0.065,0.005,0.800,0.007,61.000,35.000,100
1,20160101,종로구,0.047,0.005,0.700,0.006,68.000,51.000,100
2,20160101,용산구,0.046,0.005,0.700,0.005,59.000,46.000,100
3,20160101,은평구,0.039,0.010,0.700,0.007,58.000,35.000,101
4,20160101,서대문구,0.060,0.007,0.900,0.005,69.000,39.000,101
...,...,...,...,...,...,...,...,...,...
36514,20190908,용산구,0.020,0.031,0.446,0.004,29.833,22.667,100
36515,20190908,은평구,0.014,0.039,0.604,0.004,39.292,25.458,101
36516,20190908,종로구,0.015,0.030,0.454,0.003,32.542,20.750,100
36517,20190908,중구,0.016,0.035,0.450,0.003,33.208,19.750,100


air_day.to_csv("air_avgday.csv",encoding='euc-kr',index=False)